In [53]:
!pip install keras
!pip install tensorflow


In [54]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
sns.set()

In [55]:
dataset = pd.read_csv('Churn_Modelling.csv', index_col = 'RowNumber')
dataset.head()

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
RowNumber,,,,,,,,,,,,,
1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [56]:
X_columns = dataset.columns.tolist()[2:12]
Y_columns = dataset.columns.tolist()[-1:]
print(X_columns)
print(Y_columns)

['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']
['Exited']


In [57]:
X = dataset[X_columns].values 
Y = dataset[Y_columns].values

In [58]:
from sklearn.preprocessing import LabelEncoder
X_column_transformer = LabelEncoder()
X[:, 1] = X_column_transformer.fit_transform(X[:, 1])

In [59]:
X[:, 2] = X_column_transformer.fit_transform(X[:, 2])

In [60]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
pipeline = Pipeline(
    [
        ('Categorizer', ColumnTransformer(
            [
                ("Gender Label Encoder", OneHotEncoder(categories = 'auto', drop = 'first'), [2]),
                ("Geography Label Encoder", OneHotEncoder(categories = 'auto', drop = 'first'), [1])
            ], 
            remainder = 'passthrough', n_jobs = 1)),
        ('Normalizer', StandardScaler())
    ]
)

In [61]:
X = pipeline.fit_transform(X)

In [62]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

In [63]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Input

# Initialize the classifier
classifier = Sequential()

# Add input layer and hidden layer
classifier.add(Input(shape=(X_train.shape[1],)))  # Input layer defines the input shape
classifier.add(Dense(units=6, activation='relu'))
classifier.add(Dropout(rate=0.1))

In [64]:
#Add second layer
classifier.add(Dense(6, activation = 'relu'))
classifier.add(Dropout(rate = 0.1))

In [65]:
#Add output layer
classifier.add(Dense(1, activation = 'sigmoid'))

In [66]:
#Let us take a look at our network
classifier.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_9 (Dense)                 │ (None, 6)              │            72 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 6)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 6)              │            42 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 6)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │             7 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 121 (484.00 B)

 Trainable params: 121 (484.00 B)

 Non-trainable params: 0 (0.00 B)

In [67]:
#Optimize the weights
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [68]:
#Fitting the Neural Network
history = classifier.fit(X_train, y_train, batch_size = 32, epochs = 200, validation_split = 0.1, verbose = 2)

Epoch 1/200
225/225 - 4s - 17ms/step - accuracy: 0.6558 - loss: 0.6387 - val_accuracy: 0.8000 - val_loss: 0.4912
Epoch 2/200
225/225 - 1s - 3ms/step - accuracy: 0.8061 - loss: 0.4808 - val_accuracy: 0.8138 - val_loss: 0.4428
Epoch 3/200
225/225 - 1s - 3ms/step - accuracy: 0.8092 - loss: 0.4558 - val_accuracy: 0.8175 - val_loss: 0.4245
Epoch 4/200
225/225 - 1s - 3ms/step - accuracy: 0.8118 - loss: 0.4384 - val_accuracy: 0.8250 - val_loss: 0.4121
Epoch 5/200
225/225 - 1s - 2ms/step - accuracy: 0.8104 - loss: 0.4374 - val_accuracy: 0.8250 - val_loss: 0.4034
Epoch 6/200
225/225 - 1s - 3ms/step - accuracy: 0.8151 - loss: 0.4260 - val_accuracy: 0.8325 - val_loss: 0.3965
Epoch 7/200
225/225 - 1s - 2ms/step - accuracy: 0.8181 - loss: 0.4175 - val_accuracy: 0.8350 - val_loss: 0.3884
Epoch 8/200
225/225 - 1s - 2ms/step - accuracy: 0.8217 - loss: 0.4152 - val_accuracy: 0.8438 - val_loss: 0.3814
Epoch 9/200
225/225 - 1s - 2ms/step - accuracy: 0.8239 - loss: 0.4097 - val_accuracy: 0.8425 - val_loss

In [69]:
y_pred = classifier.predict(X_test)
print(y_pred[:5])

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
[[0.24435927]
 [0.3002689 ]
 [0.15855044]
 [0.05907806]
 [0.12165204]]


In [70]:
#Let us use confusion matrix with cutoff value as 0.5
y_pred = (y_pred > 0.5).astype(int)
print(y_pred[:5])

[[0]
 [0]
 [0]
 [0]
 [0]]


In [71]:
#Making the Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1536   59]
 [ 217  188]]


In [72]:
#Accuracy of our NN
print(((cm[0][0] + cm[1][1])* 100) / len(y_test), '% of data was classified correctly')

86.2 % of data was classified correctly
